In [ ]:
import logging

import numpy

import cicada.communicator
import cicada.encoder
import cicada.additive

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, modulus=251, precision=4)

    secret = numpy.array(3) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}", src=0)

    secret_share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=())
    
    decomposition_share = protocol.bit_decompose(secret_share)
    decomposition = protocol.reveal(decomposition_share)
    log.info(f"Player {communicator.rank} decomposition shape: {decomposition.shape}", src=1)
    log.info(f"Player {communicator.rank} decomposition: {decomposition}", src=2)

main();

Note that the decomposition contains 8 bits with the integer value "3" stored in the first four as $0 0 1 1$, and the fractional value ".0" stored in the last four as $0 0 0 0$.

To cement our understanding, let's run the same computation with something more complex than an integer:

In [ ]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, modulus=251, precision=4)

    secret = numpy.array(3.5) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}", src=0)

    secret_share = protocol.share(src=0, secret=protocol.encoder.encode(secret), shape=())
    
    decomposition_share = protocol.bit_decompose(secret_share)
    decomposition = protocol.reveal(decomposition_share)
    log.info(f"Player {communicator.rank} decomposition shape: {decomposition.shape}", src=1)
    log.info(f"Player {communicator.rank} decomposition: {decomposition}", src=2)

main();

Now we see that the integer portion of our value is still stored in the first four bits as $0011$ while the fractional portion "0.5" is stored in the last four as $1000$.